### DTS - Complete Streets

# 08 - Modal Priortization

**Author:** rmangan

---

**Purpose:**

Implement Modal Priortizaiton logic for street segments that have Unconstrained Modal Width > ROW Width estimate. 

**This script performs the following functions:**

1. Pre-process data - add all necessary fields, copy unconstrained values to new constrained fields

2. Process Boulevards

3. Process Avenues

4. Process Main Streets

5. Process Major Streets

6. Process Streets/Lanes/Alleys

7. Cleanup (if needed)

8. Results Analysis

9. Join Data to Feature Services (if needed)

3. (optional utility functions) Resent all typologies to NULL for reprocessing


**Global Assumptions and Notes:**
1. All typology comments with values in "Note" field are skipped, comments must be manually addressed in ArcGIS Pro.
2. item
3. item
4. item

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for common data geo-processing
2. arcgis 1.83 - used for connecting to AGOL org and related items
2. pandas 1.1 - (optional) used for QC purposess and data exploration

In [ ]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# (optional) login to AGOL if accessing feature services
my_gis = GIS()

print("ArcGIS Online Org account")

pw = input("Enter AGOL password...")

gis = GIS("http://dithnl.maps.arcgis.com", "ray.mangan_ch2m", pw)
print("Logged in as " + str(gis.properties.user.username))

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

modal_composite_url = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

modal_composite_url_copy = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"


In [ ]:
#Calculate Constrained Modal width

def calc_constrained(cs_type):
    #calculate all constrained modal width fields
    
    count = 0
    err_count = 0

    fields = ["SEGMENTID",                 #0
              "modal_width_ped_const",     #1
              "modal_width_bike_const",    #2
              "modal_width_auto_const",    #3
              "modal_width_bus_const",     #4
              "modal_width_park_const",    #5
              "modal_width_medians_const", #6
              "modal_width_const",         #7
              "modal_area_const",          #8
              "modal_area_diff_const",     #9
              "Fac_Type_BP",               #10
              "Fac_Type_BR",               #11
              "NewTier"]                   #12
    
    where_clause = ('"cs_type"= \'{}\''.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("row")
            try:
                #igonore shared road-way widths from bike width
                if row[10] == "Shared Roadway" or row[11] == "Shared Roadway":     
                    width = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
                
                #ignore bike shared use path for ped > 7    
                elif (row[10] == "Shared Use Path" or row[10] == "Shared Use Path") and row[1] is not None and row[1] > 7:                   
                    width = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
                
                #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
                elif (row[10] == "Shared Use Path" or row[11] == "Shared Use Path") and row[1] is not None and row[1] < 7:   
                    width = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
            
                #summarize all other segments
                else:  
                    width = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
            
                cursor.updateRow(row)
            
            except (ValueError,TypeError) as error:
                err_count += 1
                print("Error at SegmentID: {0} - {1} ".format(row[0],error))
                
    print("Constrained widths calculated for {0} items".format(count))
    print("Errors (total) for {0} items".format(err_count))
    print("Modal Widths - Constrained complete.")

In [ ]:
#Remove parking from Constrained Street Types if available, only alters Constrained Modal Width fields.
#Note this function does NOT recalculate modal width totals

def remove_parking(cs_type):
    #set constrained modal values for parking to 0 for a given street type
    #accept cs_type as an input street type variable
    
    count = 0
           
    #fields to expose to update cursor            
    fields = ["modal_width_park_const",   #0
              "modal_width_const"]        #1  
        
    where_clause = ('"cs_type"= \'{}\' AND' 
                    '"modal_width_park_const" > 0 AND'
                    '(NOT "modal_width_park_const" IS NULL) AND'
                    '("modal_width_const" > "mean_row_2020")'.format(cs_type))
        
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            try:
                row[0] = 0    
                count += 1
                cursor.updateRow(row)
            except ValueError as error:
                print(error)
                        
    print("Done - {0} records updated".format(count))

In [ ]:
def reduce_bike(cs_type):
    #Reduce Proposed Bike widths to minimum if present, set parallel route evaluation flag to 1
    #Only processes streets where parking is not availalble (see where clause in update cursor)
    #Does not reduce existing Bike Facilities

    count = 0
    
    #fields to expose to update cursor
    fields = ["Fac_Type_BP",            #0
              "Fac_Type_BR",            #1
              "modal_width_bike_const", #2
              "bike_parallel_route"]    #3
         
    where_clause = ('"cs_type" = \'{0}\' AND' 
                    '("modal_width_park_const" = 0 OR ("modal_width_park_const" IS NULL)) AND (NOT ("Fac_Type_BP" IS NULL AND "Fac_Type_BR" IS NULL)) AND'
                    '("modal_width_const" > "mean_row_2020")'.format(cs_type))
    
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("row")
            try:
                #Buffered Bike Lanes - Set to minimum 8' width and flag for parallel route evaluation
                if (row[0] == "Buffered Bike Lane" or 
                    row[1] == "Buffered Bike Lane"):
                    row[2] = 8
                    row[3] = 1
                
                #Protected Bike Lanes - Set to minimum 10' width and flag for parallel route evaluation 
                elif (row[0] == "Protected Bike Lane" or
                      row[1] == "Protected Bike Lane"):
                    row[2] = 10
                    row[3] = 1
            
                #All other bike features - Set to 5' width and flag for parallel route evaluation
                elif (row[0] == "Bike Lane" or
                      row[0] == "Climbing Lane" or
                      row[0] == "Shoulder Bikeway" or
                      row[0] == "Shared Use Path" or
                      row[0] == "Shared Roadway" or
                      row[1] == "Bike Lane" or
                      row[1] == "Climbing Lane" or
                      row[1] == "Shoulder Bikeway" or
                      row[1] == "Shared Use Path" or
                      row[1] == "Shared Roadway"):
                    row[2] = 5
                    row[3] = 1
                
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)

    print("Done - {0} records updated".format(count))      

In [ ]:
def bike_ped(cs_type):
    #reduce constrained modal widths for bike facilites to 0. (Bike facilities will be placed w/ ped faciliites)
    
    count = 0
    
    #fields to expose to update cursor
    fields = ["SEGMENTID",               #0
              "FULLNAME",                #1
              "modal_width_bike_prop",   #2 - Modal Width - Bike (Proposed - UnConstrained)
              "modal_width_bike_const",  #3 - Modal Width - Bike (Proposed - Constrained)
              "modal_width_ped_const"]   #4 - Modal Width - Ped (Proposed - Constrained)
         
    where_clause = ('"cs_type" = \'{0}\' AND'
                    ' ("modal_width_const" > "mean_row_2020") AND'
                    ' ((NOT "Fac_Type_BE" IS NULL) OR (NOT "Fac_Type_BP" IS NULL) OR (NOT "Fac_Type_BR" IS NULL))'.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            print("SegmentID: {0}, Name: {1}, Ped Const: {2}, Bike Proposed: {3}".format(row[0], row[1], row[4], row[2])) #use to verify cursor has valid query
            try:
                row[3] = 0
                
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)

    print("Done - {0} records updated".format(count))    

In [ ]:
def cs_flag(cs_type):
#Set Street Type re-evaluation flag for constrained streets
         
    count = 0

    #fields to expose to update cursor
    fields = ["cs_type_revise"] #0
    
    where_clause = ('"cs_type" = \'{0}\' AND' 
                    '("modal_width_const" > "mean_row_2020")'.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            try:
                row[0] = 1   
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)
                                          
    print("Done - {0} records updated".format(count))

## 01 - Pre-process data
---

Add all necessary fields for data procesing. Copy unconstrained modal width and area values into newly added constrained modal fields. Constrained fields will then be updated by subsquent processing sections. This will enable comparison between Unconstrained and Constrained modal values.

**This section performs the following functions:**

1. Add all necessary fields
2. Copy unnconstrained modal values into new constrained fields (to be updated in later sections)

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
#01 - add necessary fields & write all modal values to new fields
print("Adding new fields...")
#store field, field alias in dict
new_fields = {"modal_width_ped_const":      "Modal Width - Ped Constrained",
              "modal_width_bike_const":     "Modal Width - Bike Constrained",
              "modal_width_auto_const":     "Modal Width - Auto Constrained",
              "modal_width_bus_const":      "Modal Width - Bus Constrained",
              "modal_width_park_const":     "Modal Width - Parking Constrained",
              "modal_width_medians_const":  "Modal Width - Medians Constrained",
              "modal_width_const":          "Modal Width - Total Constrained",
              "modal_area_const":           "Modal Area - Total Constrained (sq. ft.)",
              "modal_area_diff_const":      "Modal Area - Total Constrained Exceedence (sq. ft.)",
              "bike_parallel_route":        "Bike - Evaluate Parallel Routes",
              "lane_count_rev":             "Lane Count (Constrained)",
              "cs_type_revise":             "Complete Street Type - Re-Evaluate"}

#get fields in current input datset
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

#loop through dict and add fields if they are not currently in the dataset
for key,value in new_fields.items():
    if key in field_names:
        print("{0} field already present, skipping".format(key))
    elif key in ("modal_area_const","modal_area_diff_const"):
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="FLOAT", field_alias = value)
    else:
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="SHORT", field_alias = value)
          
print("Field additions complete")
print("\n")

In [ ]:
#write all previous proposed modal values to new fields (values will be altered in subsquent processing)

#fields to expose to update cursor
fields = ["modal_width_auto_prop",    #0
          "modal_width_auto_const",   #1
          "modal_width_bike_prop",    #2
          "modal_width_bike_const",   #3
          "modal_width_bus_prop",     #4
          "modal_width_bus_const",    #5
          "modal_width_park_prop",    #6
          "modal_width_park_const",   #7
          "modal_width_ped_prop",     #8
          "modal_width_ped_const",    #9
          "modal_width_max",          #10
          "modal_width_const",        #11
          "modal_area_max" ,          #12     
          "modal_area_const",         #13
          "modal_area_diff",          #14
          "modal_area_diff_const"]    #15

print("Copying all Unconstrained values to Constrained fields for initial processing...")
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[1] = row[0]
            row[3] = row[2]
            row[5] = row[4]
            row[7] = row[6]
            row[9] = row[8]
            row[11] = row[10]
            row[13] = row[12]
            row[15] = row[14]         

            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            print(error)
            #err_count += 1
            #print("Error at SegmentID: {0} - {1} ".format(row[0],error))          

print("Done - Constrained Fields calculated from Unconstrained")

## 02 - Modal Prioritization - Boulevards
---

Calculate Modal Priortization logic and revised Modal widths for Boulevards.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/45a7bf56620b99717ae38310a31b367948880137/Boulevard.jpg" alt="Logic flow diagram for Boulevards" style="height: 200px;"/>

**This section performs the following functions for all Boulevard Street Types:**

1. If ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width
3. If ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Minimum Modal widths for Bike Facilities used:
   1. Buffered Bike Lane - 8'
   2. Protected Bike Lane - 10'
   3. Bike Lane, Climbing Lane, Shoulder Bikeway, Shared Use Path, Shared Roadway - 5'

In [ ]:
## 01 - remove parking

print("Processing Boulevards....")
print("02.01 - removing parking...\n")
remove_parking("Boulevard")
print("02.01 - done\n")

In [ ]:
## 02 - Re-Calcualte Constrained Modal Width

print("02.02 - Re-calculate constrained width values\n")
calc_constrained("Boulevard")
print("02.02 - done\n")

In [ ]:
## 03 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1

print("02.03 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation\n")  
reduce_bike("Boulevard")
print("02.03 - done\n")

In [ ]:
## 04 - Re-Calcualte Constrained Modal Width for Boulevards

print("02.04 - Re-calculate constrained width values\n")
calc_constrained("Boulevard")
print("02.04 - done\n")

In [ ]:
## 05 - Set all remaining constrained widths that are boulevards to street type re-evaluation

print("02.05 - Set street type re-evaluation flage for remaining constrained boulevards\n")
cs_flag("Boulevard")
print("02.05 - done\n")

print("Boulevards Complete")

## 03 - Modal Prioritization - Avenues
---

Calculate Modal Priortization logic and revised Modal widths for Boulevards.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Avenue.jpg" alt="Logic flow diagram for Avenues" style="height: 200px;"/>

**This section performs the following functions for all Avenue Street Types:**

1. If Street Type = Avenue, ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width field
3. If Street Type = Avenue, ROW = Constrained, and parking = 0 or NULL, and VOC <= 0.8, reduce auto transit lane count.
4. Re-calcualte Constrained Modal width field
5. If Street Type = Avenue, ROW = Constrained, parking = 0 or NULL, VOC > 0.8, reduce sidewalk facilities to minimum width
6. Re-calcualte Constrained Pedestrian width
7. Re-calculalate Constrained Modal width fields
8. If still constrained, set flag for street type re-evaluation


**Assumptions and Notes:**
1. Streets must have a minimum of 3 lanes to be eligible for lane count reduction, regardless of one-way/two-way classification
2. All streets have lanes counts reduced by only 1 (regardless of lane count)
3. Existing Ped Facilites from sidealk inventory are not reduced in size. Only planned ped improvements are reduced.

In [ ]:
## 01 - Remove parking

print("Processing Avenues....\n")

print("03.01 - removing parking...\n")
remove_parking("Avenue")
print("03.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("03.02 - Re-calculate constrained width values\n")
calc_constrained("Avenue")
print("03.02 - done\n")

In [ ]:
## 03 - Reduce lane count by 1 for all segments & reduce auto modal width by 12 (assume outside lane removed)

print("03.03 - Remove Auto lanes for Streets <= 0.8 VOC\n")

count = 0
    
#fields to expose to update cursor
fields = ["SEGMENTID",              #0
          "FULLNAME",               #1
          "ONEWAY",                 #2   
          "AB_LANE",                #3
          "BA_LANE",                #4
          "lanes_assumed",          #5
          "modal_width_auto_const", #6
          "lane_count_rev",         #7
          "VOC_max"]                #8
         
where_clause = ('"cs_type" = \'Avenue\' AND'
                ' ("modal_width_const" > "mean_row_2020") AND'
                ' NOT ("VOC_max" IS NULL) AND'
                ' "VOC_max" <= 0.8 AND'
                ' NOT ("AB_LANE" IS NULL OR "BA_LANE" IS NULL) AND'
                ' ("AB_LANE" + "BA_LANE" >= 3)')
    
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        print("SegmentID: {0}, Name: {1}, AB_Lane: {2}, BA_Lane: {3}, VOC: {4}".format(row[0], row[1], row[3], row[4], row[8])) #use to verify cursor has valid query
        try:
            lane_current = (row[3] + row[4])
            lane_revision = lane_current - 1
            auto_current = row[6]
            auto_revised = (row[6] - 12)
            
            print("Current lanes: {}".format(lane_current))
            print("Revised lanes: {}".format(lane_revision))
            print("Current Auto width: {}".format(auto_current))
            print("Revised Auto width: {}".format(auto_revised))
            
            row[5] = lane_revision
            row[6] = auto_revised
                
            #cursor.updateRow(row)
            count += 1
        
        except ValueError as error:
            print(error)

print("Done - {0} records updated".format(count))    
print("03.03 - done\n")

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("03.04 - Re-calculate constrained width values\n")
calc_constrained("Avenue")
print("03.04 - done\n")

In [ ]:
## 05 - Reduce sidewalk widths to minimum

print("03.05 - Reduce sidewalk widths to minimum \n")

count = 0
    
#fields to expose to update cursor
fields = ["SEGMENTID",              #0
          "FULLNAME",               #1
          "modal_width_ped_exist",  #2 - Modal Width - Ped (Existing)
          "modal_width_ped_prop",   #3 - Modal Width - Ped (Proposed)
          "modal_width_ped_const"]  #4 - Modal Width - Ped (Proposed - Constrained)
         
where_clause = ('"cs_type" = \'Avenue\' AND'
                ' ("modal_width_const" > "mean_row_2020") AND'
                ' ((NOT "FID_Ped_Add_temp" IS NULL) OR (NOT "FID_Ped_Improve_temp" IS NULL))')

with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        print("SegmentID: {0}, Name: {1}, Ped Uncon: {2}, Ped Con: {3}".format(row[0], row[1], row[2], row[3])) #use to verify cursor has valid query
        try:
            #change all proposed sidewalk modal values to 8' total ped width value if Unconstrained was increased due to Ped Add/Ped Improve
            if row[3] > row[2]:
                row[4] = 8
                count += 1
        
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)


print("Done - {0} records updated".format(count))    
print("03.05 - done\n")

In [ ]:
## 06 - Re-Calculate Constrained Modal Width

print("03.06 - Re-calculate constrained width values\n")
calc_constrained("Avenue")
print("03.06 - done\n")

In [ ]:
## 07 - Set all remaining constrained widths to street type re-evaluation

print("03.07 - Set street type re-evaluation flage for remaining constrained segments\n")
cs_flag("Avenue")
print("03.07 - done\n")

print("Avenues Complete")

## 04 - Modal Priortization - Main Streets
---

Calculate Modal Priortization logic and revised Modal widths for Main Strets.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/MainStreet.jpg" alt="Logic flow diagram for Main Streets" style="height: 200px;"/>

**This section performs the following functions for all 'Main Street' Street Types:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking > 0 or NULL, remove parking
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Combingin bike & pedestrain facilites is done by...
2. All streets have lanes counts reduced by only 1 (regardless of lane count)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
## 01 - Combine Bike & Pedestrian Facilities

print("Processing Main Streets....\n")

print("04.01 - Combining Bike & Pedestrian facilities...\n")
bike_ped("Main Street")
print("04.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("04.02 - Re-calculate constrained width values\n")
calc_constrained("Main Street")
print("04.02 - done\n")

In [ ]:
## 03 - Remove parking

print("Processing Main Streets....\n")

print("04.03 - removing parking...\n")
remove_parking("Main Street")
print("04.03 - done\n")

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("04.04 - Re-calculate constrained width values\n")
calc_constrained("Main Street")
print("04.04 - done\n")

In [ ]:
## 05 - Set all remaining constrained widths to street type re-evaluation

print("04.05 - Set street type re-evaluation flage for remaining constrained segments\n")
cs_flag("Main Street")
print("04.05 - done\n")

print("Main Streets Complete")

## 05 - Modal Priortization - Major Streets
---

Calculate Modal Priortization logic and revised Modal widths for Major Streets.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/MajorStreet.jpg" alt="Logic flow diagram for Major Streets" style="height: 200px;"/>

**This section performs the following functions for all 'Major Street' Street Types:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking > 0 or NULL, remove parking
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Logic is identical to logic used for Boulevards but maintained as seperate code sections to allow for individual changes if needed
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
## 01 - Remove parking

print("Processing Major Streets....\n")

print("05.01 - removing parking...\n")
remove_parking("Major Street")
print("05.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("05.02 - Re-calculate constrained width values\n")
calc_constrained("Major Street")
print("05.02 - done\n")

In [ ]:
## 03 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1

print("05.03 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation\n")  
reduce_bike("Major Street")
print("05.03 - done\n")

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("05.04 - Re-calculate constrained width values\n")
calc_constrained("Major Street")
print("05.04 - done\n")

In [ ]:
## 05 - Set all remaining constrained widths to street type re-evaluation

print("02.05 - Set street type re-evaluation flage for remaining constrained boulevards\n")
cs_flag("Boulevard")
print("02.05 - done\n")

print("Major Streets Complete")

## 06 - Modal Priortization - Streets, Lanes, Alleys
---

Calculate Modal Priortization logic and revised Modal widths for Street and Lane/Alley

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Street.jpg" alt="Logic flow diagram for Streets and Lane/Alleys" style="height: 200px;"/>

**This section performs the following functions for all 'Street' and Lane/Alley Street Types:**

1. If ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width
3. If ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
6. Re-calcualte Constrained Modal width field
7. If ROW = Constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
## 01 - Remove parking

print("Processing Streets, Lanes, and Alleys....\n")

print("06.01 - removing parking...\n")
remove_parking("Street")
print("06.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("06.02 - Re-calculate constrained width values\n")
calc_constrained("Street")
print("06.02 - done\n")

In [ ]:
## 03 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1

print("06.03 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation\n")  
reduce_bike("Street")
print("06.03 - done\n")

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("06.04 - Re-calculate constrained width values\n")
calc_constrained("Street")
print("06.04 - done\n")

In [ ]:
## 05 - Combine Bike & Pedestrian Facilities

print("06.05 - Combining Bike & Pedestrian facilities...\n")
bike_ped("Street")
print("06.05 - done\n")

In [ ]:
## 06 - Re-Calculate Constrained Modal Width

print("06.06 - Re-calculate constrained width values\n")
calc_constrained("Street")
print("06.06 - done\n")

In [ ]:
## 07 - Set all remaining constrained widths to street type re-evaluation

print("06.07 - Set street type re-evaluation flage for remaining constrained boulevards\n")
cs_flag("Boulevard")
print("02.07 - done\n")

print("Streets, Lanes/Alleys done\n")

In [ ]:
print("Modal Prioritizitation processing complete")

## 07 - Results Analysis/ QC
---

Calculate Modal Priortization logic and revised Modal widths for Street and Lane/Alley



**This section performs the following functions for all 'Street' and Lane/Alley Street Types:**

1. If ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width
3. If ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
6. Re-calcualte Constrained Modal width field
7. If ROW = Constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
#get fields in current input datset
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

"SEGMENTID" in field_names

const_fields = ["SEGMENTID",                 #0
                "modal_width_ped_const",     #1
                "modal_width_bike_const",    #2
                "modal_width_auto_const",    #3
                "modal_width_bus_const",     #4
                "modal_width_park_const",    #5
                "modal_width_medians_const", #6
                "modal_width_const",         #7
                "modal_area_const",          #8
                "modal_area_diff_const"      #9
                "Fac_Type_BP",               #10
                "Fac_Type_BR",               #11
                "NewTier"]                   #12

for new_fields in const_fields:
    print(new_fields in field_names)

In [ ]:
#Create DF for CSDM analysis
modal_df = pd.DataFrame.spatial.from_featureclass(modal_composite)

modal_df_fields = modal_df.columns.values.tolist()

modal_df_fields.sort()

#print all fields for reference
for i in modal_df_fields:
    print(i)

In [ ]:
#Fields for Dataframe
# df_fields = ["SEGMENTID",
#              "FULLNAME",
#              "CS_type",
#              "ONEWAY",
#              "AB_LANE",
#              "BA_LANE",
#              "thrulanes",
#              "ln_exist_num",
#              "ln_prop_num",
#              "lanes_assumed",
#              "mean_row_2020",
#              "row_exist_num",
#              "row_prop_num",
#              "modal_width_ped_exist",
#              "modal_width_ped_prop",
#              "modal_width_bike_prop",
#              "VOC_max",
#              "modal_width_max"]

# df_fields = ["SEGMENTID",
#              "FULLNAME",
#              "CS_type",
#              "ONEWAY",
#              "mean_row_2020",
#              "Tier",
#              "NewTier",
#              "modal_width_ped_exist",
#              "modal_width_ped_prop",
#              "modal_width_bike_prop",
#              "modal_width_const",
#              "modal_width_max"]


df_fields = ["SEGMENTID",
             "FULLNAME",
             "CS_type",
             "ONEWAY",
             "mean_row_2020",
             "Tier",
             "NewTier",
             "Fac_Type_BE",
             "Fac_Type_BP",
             "Fac_Type_BR",
             "modal_width_ped_exist",
             "modal_width_ped_prop",
             "modal_width_bike_exist",
             "modal_width_bike_prop",
             "modal_width_const",
             "modal_width_max"]



#Pre-set Filters for Dataframe

filt_boulevard = ((modal_df["CS_type"] == "Boulevard" ))

filt_avenue = ((modal_df["CS_type"] == "Avenue" ))

filt_main_street = ((modal_df["CS_type"] == "Main Street" ))

filt_street = ((modal_df["CS_type"] == "Street" ))

filt_lane = ((modal_df["CS_type"] == "Lane/Alley" ))

filt_mall = ((modal_df["CS_type"] == "Mall" ))

filt_constrained = ((modal_df["mean_row_2020"] < modal_df["modal_width_max"] ))


filt_parking = ((modal_df["modal_width_park_exist"] > 0))

filt_bike = ((modal_df["modal_width_bike_prop"] > 0))

filt_VOC = ((modal_df["VOC_max"] <= 0.8))

filt_lanes = ((modal_df["AB_LANE"] <= 0.8))

filt_row = ((modal_df["mean_row_2020"] < 70 ))

filt_cs_examples = ((~modal_df["CS_example_num"].isnull()))

filt_ped_improve = ((~modal_df["FID_Ped_Improve_temp"].isnull()))

filt_ped_add = ((~modal_df["FID_Ped_Add_temp"].isnull()))


filt_example = (((modal_df["AB_SPEED"] > 35 ) |
             (modal_df["BA_SPEED"] > 35 )) &
             (modal_df["TOT_FLOW_D"] > 10000) &
             ((modal_df["AB_LANE"] + modal_df["BA_LANE"]) >= 4 )
             & ((~modal_df["CS_example_num"].isnull()))  
            )

filt_change = (((modal_df["AB_SPEED"] < 35 ) |
             (modal_df["BA_SPEED"] < 35 )) &
             (modal_df["TOT_FLOW_D"] > 10000) &
             ((modal_df["AB_LANE"] + modal_df["BA_LANE"]) <= 4 )
             #& ((~cs_df["CS_example_num"].isnull()))  
            )

In [ ]:
#pd.set_option('display.max_rows', 250)

#cs_df.loc[filt35_CS,df_fields].sort_values(by="FULLNAME")

#cs_df.filter(df_fields)

#Parking
#modal_df.loc[(filt_constrained & filt_avenue & filt_VOC), df_fields]

#Boulevards
#modal_df.loc[(modal_df["CS_type"] == "Boulevard"),df_fields]

#Avenues
#modal_df.loc[(modal_df["CS_type"] == "Avenue"),df_fields]

#Main Streets
#modal_df.loc[(modal_df["CS_type"] == "Main Street"),df_fields]


modal_df.loc[filt_lane & filt_constrained, df_fields]

# modal_df.loc[filt_constrained & filt_avenue & (filt_ped_add), df_fields]